In [49]:
import types
import json
from jinja2 import Template

class StackLang:
    def __init__(self):
        self.stack = []

    def push(self, value):
        self.stack.append(value)

    def pop(self):
        if self.stack:
            return self.stack.pop()
        raise IndexError("Pop from an empty stack")

    def MAKE_PROMPT(self):
        if len(self.stack) >= 2:
            prompt_template = self.pop()
            params = self.pop()
            template = Template(prompt_template)
            self.push(template.render(params))

        else:
            raise IndexError("CALL-ing a model requires 2 params: prompt and model params.")

    def CALL(self):
        if len(self.stack) >= 2:
            model = self.pop()
            prompt = self.pop()
            print('%% Calling model %s with prompt: %s' %(model, prompt))
        else:
            raise IndexError("CALL-ing a model requires 2 params: prompt and model params.")

    def execute(self, commands):
        #print(self)

        for cmd in commands:
            print(self)
            if cmd.startswith('"""'):
                self.push(cmd)
            elif cmd.startswith('{'):
                self.push(json.loads(cmd))
            else:
                if hasattr(self, cmd):
                    getattr(self, cmd)()
                else:
                    raise ValueError(f"Unknown command: {cmd}")

    def __str__(self):
            return str(self.stack)



lang = StackLang()
commands = ['{"friend": "Arnaud"}', 
            '"""how are you doing {{friend}}"""',
            "MAKE_PROMPT",
            '{"model": "gpt-3.5", "temperature": 0}',
            'CALL' ]
lang.execute(commands)
print(lang)  # Output should be [14]

[]
[{'friend': 'Arnaud'}]
[{'friend': 'Arnaud'}, '"""how are you doing {{friend}}"""']
['"""how are you doing Arnaud"""']
['"""how are you doing Arnaud"""', {'model': 'gpt-3.5', 'temperature': 0}]
% Calling model {'model': 'gpt-3.5', 'temperature': 0} with prompt: """how are you doing Arnaud"""
[]


In [44]:
def process_file(filename):
    lines = open(filename).readlines()
    clean_lines = []
    i = 0
    while i < len(lines):
        if lines[i].startswith('"""') is False:
            clean_lines.append(lines[i].strip())
            i = i + 1
        else:
            j = i
            while (j < len(lines) and lines[j].endswith('"""\n') is False):
                j = j + 1
            # clean_lines.append("\n".join(  [k[:-1] for k in lines[i:j+1] ] ) )
            clean_lines.append("\n".join(lines[i:j+1]).strip())
            i = j + 1
    return [k for k in clean_lines if k != '']

In [50]:
lang = StackLang()

commands = process_file('llm1.txt')

lang.execute(commands)
print(lang)


[]
[{'friend': 'Arnaud'}]
[{'friend': 'Arnaud'}, '"""how\n\n    are\n\n\n\n    you doing {{friend}}"""']
['"""how\n\n    are\n\n\n\n    you doing Arnaud"""']
['"""how\n\n    are\n\n\n\n    you doing Arnaud"""', {'model': 'gpt-3.5', 'temperature': 0}]
% Calling model {'model': 'gpt-3.5', 'temperature': 0} with prompt: """how

    are



    you doing Arnaud"""
[]
